In [1]:
import findspark
findspark.init()
# Creating Spark Context
from pyspark import SparkContext
from pyspark.sql.session import SparkSession
sc = SparkContext("local", "first app")
spark = SparkSession(sc)
import plotly.graph_objects as go
from pyspark.sql import SQLContext
sqlContext = SQLContext(sc)

In [2]:
text_file = sqlContext.read.format('com.databricks.spark.csv').options(header='true', inferschema='true', quote='"', delimiter=',').load("C:\\Users\\mini_\\Desktop\Practica-Semi2\\archivos\\TraficoAereoGt.csv")
rddfiltro = text_file.rdd.map(tuple)

In [3]:
rddAERO= rddfiltro.map(lambda word: (word[0].split('/')[1],word[5]))
rddAERO.take(10)

[('01', 195),
 ('02', 112),
 ('03', 159),
 ('04', 193),
 ('05', 82),
 ('06', 143),
 ('07', 91),
 ('08', 196),
 ('09', 176),
 ('10', 119)]

In [4]:
rddCONT=rddAERO.reduceByKey(lambda a,b: a+b)
print("Conteo total -> %s" % rddCONT.collect())

Conteo total -> [('01', 6034), ('02', 5912), ('03', 5600), ('04', 5723), ('05', 5690), ('06', 5476), ('07', 5972), ('08', 5850), ('09', 5425), ('10', 5347), ('11', 6467), ('12', 5862)]


In [5]:
#Top 3 meses con mayor número de pasajeros
rddORDEN= sc.parallelize(rddCONT.sortBy(lambda a: a[1],False).take(3))
print("Top 3 meses con mayor número de pasajeros de salida%s"%rddORDEN.collect())

Top 3 meses con mayor número de pasajeros de salida[('11', 6467), ('01', 6034), ('07', 5972)]


In [6]:
#Separando nombres y totales
#Vista de datos
rddNombres = rddORDEN.map(lambda x: (x[0]))
rddTotales = rddORDEN.map(lambda x: (x[1]))
print(rddNombres.collect())
print(rddTotales.collect())

['11', '01', '07']
[6467, 6034, 5972]


In [7]:
#Graficando pie
fig = go.Figure(data=go.Pie(labels=rddNombres.collect(),values=rddTotales.collect()))
fig.update_layout(title_text='3 primeros meses con mayor número de pasajeros de salida.',title_font_size=30)
fig.update_traces(hoverinfo='label+percent', textinfo='value', textfont_size=20)
fig.write_html('R2_A2.html', auto_open=True)